# Generación de Reporte

Este notebook genera el reporte en Markdown con:
- KPIs y definiciones
- Tablas de análisis
- Contexto y conclusiones

In [ ]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq
from datetime import datetime
from tabulate import tabulate

# Configuración de rutas
PROJECT_ROOT = Path().resolve().parent
OUTPUT_DIR = PROJECT_ROOT / "output"
PARQUET_DIR = OUTPUT_DIR / "parquet"

# Leer datos limpios
ventas_df = pd.concat([
    pd.read_parquet(f) 
    for f in PARQUET_DIR.glob('ventas_clean/**/*.parquet')
])

In [ ]:
# Calcular KPIs
kpis = {
    'total_ventas': ventas_df['importe_total'].sum(),
    'num_transacciones': len(ventas_df),
    'ticket_promedio': ventas_df['importe_total'].mean(),
    'clientes_unicos': ventas_df['id_cliente'].nunique(),
    'productos_unicos': ventas_df['id_producto'].nunique()
}

# Top productos por ventas
top_productos = (
    ventas_df.groupby('id_producto')
    .agg({
        'importe_total': 'sum',
        'unidades': 'sum'
    })
    .sort_values('importe_total', ascending=False)
    .head(5)
)

# Ventas por día
ventas_diarias = (
    ventas_df.groupby('fecha')
    .agg({
        'importe_total': 'sum',
        'id_cliente': 'nunique'
    })
    .rename(columns={'id_cliente': 'clientes_unicos'})
)

# Generar reporte en Markdown
report = f"""# Reporte de Ventas

## KPIs
- **Ventas Totales**: €{kpis['total_ventas']:,.2f}
  - *Definición*: Suma total de importes de todas las transacciones
- **Número de Transacciones**: {kpis['num_transacciones']:,}
  - *Definición*: Total de ventas realizadas
- **Ticket Promedio**: €{kpis['ticket_promedio']:,.2f}
  - *Definición*: Importe promedio por transacción
- **Clientes Únicos**: {kpis['clientes_unicos']:,}
  - *Definición*: Número total de clientes distintos
- **Productos Únicos**: {kpis['productos_unicos']:,}
  - *Definición*: Número total de productos distintos vendidos

## Top 5 Productos por Ventas
{tabulate(top_productos, headers='keys', tablefmt='pipe', floatfmt='.2f')}

## Ventas Diarias
{tabulate(ventas_diarias, headers='keys', tablefmt='pipe', floatfmt='.2f')}

## Contexto
- **Fuente de Datos**: Archivos CSV en carpeta data/drops
- **Periodo**: {ventas_df['fecha'].min().strftime('%Y-%m-%d')} a {ventas_df['fecha'].max().strftime('%Y-%m-%d')}
- **Última Actualización**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Conclusiones
1. La venta promedio por transacción es de €{kpis['ticket_promedio']:.2f}
2. Se han registrado ventas de {kpis['productos_unicos']} productos diferentes
3. Han realizado compras {kpis['clientes_unicos']} clientes únicos

## Acciones Recomendadas
1. Analizar productos más vendidos para optimizar inventario
2. Monitorear patrones de ventas diarias para ajustar operaciones
3. Evaluar base de clientes para identificar oportunidades de fidelización
"""

# Guardar reporte
with open(OUTPUT_DIR / 'reporte.md', 'w', encoding='utf-8') as f:
    f.write(report)

print("Reporte generado en:", OUTPUT_DIR / 'reporte.md')